In [1]:
import torch #provides a wide range of algorithms for deep learning
import joblib #provides lightweight pipelining/transparent disk-caching of functions and lazy re-evaluation (memoize pattern)
import torch.nn as nn #helps in creating and training of the neural network
import numpy as np #operations on arrays
from cv2 import cv2 #read and preprocess images
import torch.nn.functional as F #applies a xD(ex. 2D) convolution over an input image composed of several input planes
import time #provides many ways of representing time in code
import import_ipynb #defines a “notebook loader” that allows you to ‘import’ other ipynb files into your current ipynb file
import CNN #the created model

importing Jupyter notebook from CNN.ipynb
Loading label binarizer...


In [2]:
# load label binarizer
lb = joblib.load('D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\lb.pkl')

In [3]:
device = torch.device('cude' if torch.cuda.is_available() else 'cpu')

In [4]:
model = CNN.CustomCNN().to(device)
model.load_state_dict(torch.load('D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\model.pth'))
print(model)
print('Model loaded')

CustomCNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=58, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Model loaded


In [5]:
def hand_area(img):
    hand = img[100:324, 100:324]
    hand = cv2.resize(hand, (224,224))
    return hand

In [12]:
cap = cv2.VideoCapture(0)
if (cap.isOpened() == False):
    print('Error while trying to open camera. Plese check again...')
# get the frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
# define codec and create VideoWriter object
out = cv2.VideoWriter('D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\msl.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width,frame_height))

In [13]:
# read until end of video
while(cap.isOpened()):
    # capture each frame of the video
    time.sleep(3)
    ret, frame = cap.read()
    # get the hand area on the video capture screen
    cv2.rectangle(frame, (100, 100), (324, 324), (20, 34, 255), 2)
    hand = hand_area(frame)
    image = hand
    
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    image = torch.tensor(image, dtype=torch.float).to(device)
    image = image.unsqueeze(0)
    
    outputs = model(image)
    _, preds = torch.max(outputs.data, 1)
    
    cv2.putText(frame, lb.classes_[preds], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    cv2.imshow('image', frame)
    out.write(frame)
    # press `q` to exit
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
# release VideoCapture()
cap.release()
# close all frames and video windows
cv2.destroyAllWindows()